In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=RuntimeWarning, module="hmmlearn")

configs = [
    dict(symbol='XAUUSD', timeframe='H1', direction='buy',
         train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
         test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
         search_type='clusters', search_subtype='simple',
         n_trials=500, n_models=10, n_jobs=-1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='buy',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='clusters', search_subtype='advanced',
    #      n_trials=500, n_models=5, n_jobs=-1),
    dict(symbol='XAUUSD', timeframe='H1', direction='buy',
         train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
         test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
         search_type='markov', search_subtype='simple',
         n_trials=500, n_models=10, n_jobs=-1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='buy',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='markov', search_subtype='advanced',
    #      n_trials=10, n_models=5, n_jobs=-1),
]

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    # cfg["n_jobs"] = cores_per_config
    cfg["tag"]    = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['search_type']} {cfg['search_subtype']}"

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")